<a href="https://colab.research.google.com/github/kumahag/Capstone-Project/blob/main/Movie_Review_Sentiment_Classification_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

True

In [2]:
# Install required dependencies
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
The following package was automat

In [4]:
import os
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Capstone Project

Mounted at /content/drive
/content/drive/MyDrive/Capstone Project


In [6]:
# Load the  train dataset and display some values
train_data =pd.read_csv("/content/drive/MyDrive/Capstone Project/Train.csv")
train_data

,review_file,content,sentiment
0,3471_8.txt,Recently shown on cable tv the movie opens wit...,positive
1,9693_8.txt,I was very surprised with this film. I was tou...,positive
2,10801_1.txt,"Now, I'm one to watch movies that got poor rev...",negative
3,9592_8.txt,"This film came out 12 years years ago, and was...",positive
4,8514_7.txt,"When an orphanage manager goes on vacation, hi...",positive
...,...,...,...
24995,2791_3.txt,"As with most of the reviewers, I saw this on S...",negative
24996,644_9.txt,A have a female friend who is currently being ...,positive
24997,4921_8.txt,Like A Streetcar Named Desire (also directed b...,positive
24998,5791_1.txt,"As a Native film professor, I can honestly say...",negative


In [7]:
#checking missing values
train_data.isna().sum()

review_file    0
content        0
sentiment      0
dtype: int64

In [8]:
#rename columns (contents & sentiment) to text and label
train_data = train_data.rename(columns = {'content':'text','sentiment':'label'})
train_data


,review_file,text,label
0,3471_8.txt,Recently shown on cable tv the movie opens wit...,positive
1,9693_8.txt,I was very surprised with this film. I was tou...,positive
2,10801_1.txt,"Now, I'm one to watch movies that got poor rev...",negative
3,9592_8.txt,"This film came out 12 years years ago, and was...",positive
4,8514_7.txt,"When an orphanage manager goes on vacation, hi...",positive
...,...,...,...
24995,2791_3.txt,"As with most of the reviewers, I saw this on S...",negative
24996,644_9.txt,A have a female friend who is currently being ...,positive
24997,4921_8.txt,Like A Streetcar Named Desire (also directed b...,positive
24998,5791_1.txt,"As a Native film professor, I can honestly say...",negative


In [9]:
# Distribution of review sentiments
train_data["label"].value_counts()


positive    12500
negative    12500
Name: label, dtype: int64

In [10]:
# Encoding the categorical (label) column
two_unique_vals = ['label']

for column in two_unique_vals:
    train_data[column] = train_data[column].replace({"positive": 1, "negative":-1})
train_data.head()

,review_file,text,label
0,3471_8.txt,Recently shown on cable tv the movie opens wit...,1
1,9693_8.txt,I was very surprised with this film. I was tou...,1
2,10801_1.txt,"Now, I'm one to watch movies that got poor rev...",-1
3,9592_8.txt,"This film came out 12 years years ago, and was...",1
4,8514_7.txt,"When an orphanage manager goes on vacation, hi...",1


In [11]:
# Split the train data => {train, eval}
train, eval = train_test_split(train_data, test_size=0.2, random_state=42, stratify=train_data['label'])
     
     

In [12]:
train_data.head()

,review_file,text,label
0,3471_8.txt,Recently shown on cable tv the movie opens wit...,1
1,9693_8.txt,I was very surprised with this film. I was tou...,1
2,10801_1.txt,"Now, I'm one to watch movies that got poor rev...",-1
3,9592_8.txt,"This film came out 12 years years ago, and was...",1
4,8514_7.txt,"When an orphanage manager goes on vacation, hi...",1


In [13]:
eval.head()

,review_file,text,label
6106,852_7.txt,THE MATADOR is hit-man movie lite....if you ca...,1
12473,10234_10.txt,"<br /><br />""Burning Paradise"" is a combinatio...",1
17907,7814_4.txt,"After a long period in the space, looking for ...",-1
8153,11008_1.txt,I bought this at tower records after seeing th...,-1
5431,3126_10.txt,I don't hand out ten star ratings easily. A mo...,1


In [14]:

print(f"new dataframe shapes: train is {train_data.shape}, eval is {eval.shape}")

new dataframe shapes: train is (25000, 3), eval is (5000, 3)


Sentiment Classification and Model Fine-Tuning

In [15]:
#instantiate tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased', num_labels =2)


In [16]:
# Save splitted subsets
train_data.to_csv("train_subset.csv", index=False)
eval.to_csv("eval_subset.csv", index=False)

In [17]:

dataset = load_dataset('csv',
                        data_files={'train':'train_subset.csv',
                        'eval':'eval_subset.csv'}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9ec89bc80b19cf3a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
#define helper functions
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 1: #'Positive'
        num = 1

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['text'], padding='max_length',truncation=True, max_length = 256)
     



In [19]:
# Change the labels to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = ['review_file', 'label','text']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 25000
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

Fine Tuning the bert-large- uncased Model




In [21]:
# define training arguments
from transformers import TrainingArguments

# Configure the trianing parameters like `num_train_epochs`: 
# the number of time the model will repeat the training loop over the dataset
training_args = TrainingArguments("test_trainer", num_train_epochs=3, load_best_model_at_end=True, evaluation_strategy="steps",
    save_strategy="steps")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
#instantiate training and evaluation sets
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['eval'].shuffle(seed=10)

In [23]:
#load pretrained model
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of sentiments in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained('bert-large-uncased',num_labels =2)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [24]:

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-24-75db7450f822>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
#converting training data to PyTorch tensors to speed up training and adding padding:
from transformers import Trainer
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Instantiate the trainer
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset,data_collator=data_collator,tokenizer=tokenizer
)
# Launch the learning process: training 
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9375
  Number of trainable parameters = 335143938
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.648400,0.694078
1000,0.709600,0.693549
1500,0.713500,0.695669
2000,0.706600,0.694250


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_tra

In [ ]:
predictions = trainer.predict(tokenized_datasets["eval_dataset"])
print(predictions.predictions.shape, predictions.labels.shape)

import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.labels)

In [ ]:

#Check to see files saved in drive
for file in os.listdir():
  print(file)